In [2]:
import pprint
import pandas as pd

# 1
df = pd.read_excel('data/data_5710_20220716.xlsx', index_col=0)

df.index = pd.to_datetime(df.index)
df = df.sort_index()
#print(df, '\n')


# 2

df = pd.read_excel('data/data_5710_20220716.xlsx', parse_dates=['일자'])
df = df.sort_values('일자')
#print(df, '\n')


#print(df['일자'].dtype)
#print(type(df['일자'].iloc[0]))

# Groupby 월봉 OHCL
df = df[['일자', '시가', '저가', '고가', '종가', '거래량']].copy()
df['year'] = df['일자'].dt.year
df['month'] = df['일자'].dt.month
#print(df.head())
#print(df['year'])
#print(df['month'])

gb = df.groupby(['year', 'month'])
#print("==>> gb.get_group((2021, 9)).head(): \n", gb.get_group((2021, 9)).head())

how = {
    '시가': 'first',
    '고가': max,
    '저가': min,
    '종가': 'last',
    '거래량': sum
}

print("==>> gb.agg(how): \n", gb.agg(how))

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [ ]:
df.groupby(pd.Grouper(key='일자', freq='m')).agg(how)


,시가,고가,저가,종가,거래량
일자,,,,,
2021-07-31,162000.0,162000.0,142500.0,147000.0,52762602.0
2021-08-31,147000.0,155000.0,141000.0,155000.0,54240960.0
2021-09-30,155000.0,157500.0,114000.0,118000.0,122697819.0
2021-10-31,118000.0,130500.0,110500.0,125500.0,57516310.0
2021-11-30,126000.0,131000.0,121500.0,122000.0,50391276.0
2021-12-31,121500.0,124000.0,112500.0,112500.0,34418041.0
2022-01-31,113500.0,117000.0,82200.0,85000.0,67362020.0
2022-02-28,87100.0,95000.0,84500.0,94100.0,49646154.0
2022-03-31,94400.0,110000.0,89600.0,106500.0,48471677.0


### 월봉 OHCLV

In [ ]:
import pandas as pd
df = pd.read_excel('data/data_5710_20220716.xlsx', index_col=0)
df.index = pd.to_datetime(df.index)
df = df.sort_index()

df['거래량'].shift(1)

c:\Users\sms20\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


일자
2021-07-15          NaN
2021-07-16    3821843.0
2021-07-19    3423188.0
2021-07-20    3475792.0
2021-07-21    6652856.0
                ...    
2022-07-11    1418763.0
2022-07-12     979002.0
2022-07-13    1082539.0
2022-07-14    1156844.0
2022-07-15    1597400.0
Name: 거래량, Length: 247, dtype: float64

In [ ]:
df['전일거래량'] = df['거래량'].shift(1)
df[['거래량', '전일거래량']]

,거래량,전일거래량
일자,,
2021-07-15,3821843.0,NaN
2021-07-16,3423188.0,3821843.0
2021-07-19,3475792.0,3423188.0
2021-07-20,6652856.0,3475792.0
2021-07-21,10409836.0,6652856.0
...,...,...
2022-07-11,979002.0,1418763.0
2022-07-12,1082539.0,979002.0
2022-07-13,1156844.0,1082539.0


In [ ]:
cond = df['거래량'] > df['전일거래량']
cond

일자
2021-07-15    False
2021-07-16    False
2021-07-19     True
2021-07-20     True
2021-07-21     True
              ...  
2022-07-11    False
2022-07-12     True
2022-07-13     True
2022-07-14     True
2022-07-15     True
Length: 247, dtype: bool

In [ ]:
print('상승일:', len(df[cond]))
print('영업일:', len(df))

상승일: 115
영업일: 247


In [ ]:
df['거래량'].diff()

일자
2021-07-15          NaN
2021-07-16    -398655.0
2021-07-19      52604.0
2021-07-20    3177064.0
2021-07-21    3756980.0
                ...    
2022-07-11    -439761.0
2022-07-12     103537.0
2022-07-13      74305.0
2022-07-14     440556.0
2022-07-15       7889.0
Name: 거래량, Length: 247, dtype: float64

In [ ]:
cond = df['거래량'].diff() > 0
거래량증가 = df['거래량'].diff() > 0
len(df[cond])

115

In [ ]:
abs_momentum = df['종가'] / df['종가'].shift(6)
cond = (abs_momentum >= 1.02) & 거래량증가
len(df[cond])

26

In [ ]:
cond.shift(1)

일자
2021-07-15      NaN
2021-07-16    False
2021-07-19    False
2021-07-20    False
2021-07-21    False
              ...  
2022-07-11    False
2022-07-12    False
2022-07-13     True
2022-07-14    False
2022-07-15    False
Length: 247, dtype: object

In [ ]:
cond_modified = cond.shift(1).fillna(False)
s = df.loc[cond_modified, '종가'] / df.loc[cond_modified, '시가']
print(s.cumprod().iloc[-1])

0.975227025173814


### 04) 이동평균

In [ ]:
import pandas as pd

df = pd.read_excel('data/ss_ex_1.xlsx', index_col=0)
df.index = pd.to_datetime(df.index)
df = df.sort_index()[['종가']]

df['종가D-1'] = df['종가'].shift(1)
df['종가D-2'] = df['종가'].shift(2)
df['ma_3'] = (df['종가'] + df['종가D-1'] + df['종가D-2'])/3
df.head()

c:\Users\sms20\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,종가,종가D-1,종가D-2,ma_3
일자,,,,
2021-02-15,84200.0,NaN,NaN,NaN
2021-02-16,84900.0,84200.0,NaN,NaN
2021-02-17,83200.0,84900.0,84200.0,84100.000000
2021-02-18,82100.0,83200.0,84900.0,83400.000000
2021-02-19,82600.0,82100.0,83200.0,82633.333333


In [ ]:
# 일반화
df['ma_3_rolling'] = df['종가'].rolling(3).mean()
df

,종가,종가D-1,종가D-2,ma_3,ma_3_rolling
일자,,,,,
2021-02-15,84200.0,NaN,NaN,NaN,NaN
2021-02-16,84900.0,84200.0,NaN,NaN,NaN
2021-02-17,83200.0,84900.0,84200.0,84100.000000,84100.000000
2021-02-18,82100.0,83200.0,84900.0,83400.000000,83400.000000
2021-02-19,82600.0,82100.0,83200.0,82633.333333,82633.333333
...,...,...,...,...,...
2021-08-09,81500.0,81500.0,82100.0,81700.000000,81700.000000
2021-08-10,80200.0,81500.0,81500.0,81066.666667,81066.666667
2021-08-11,78500.0,80200.0,81500.0,80066.666667,80066.666667


In [1]:
import pandas as pd
df = pd.read_excel('data/ss_ex_1.xlsx', index_col=0)
df.index = pd.to_datetime(df.index)

df['ma5'] = df['종가'].shift(1).rolling(5).mean()
cond = df['ma5'] < df['시가']
print(f'상승일: {len(df[cond])}')
print('영업일:', len(df))
print()
print(df[cond])

상승일: 76
영업일: 127

               종가    대비   등락률     시가     고가     저가       거래량           거래대금  \
일자                                                                            
2021-08-06  81500  -600 -0.73  81900  82500  81300  13342623  1090225883400   
2021-08-05  82100  -800 -0.97  83300  83300  82000  18485469  1526103630900   
2021-08-04  82900  1500  1.84  82200  83100  81800  25642368  2117643006850   
2021-07-26  78800  -500 -0.63  79400  79500  78800  10040975   792954940200   
2021-07-23  79300  -400 -0.50  79700  79900  79200   9098910   723397885611   
...           ...   ...   ...    ...    ...    ...       ...            ...   
2021-02-22  82200  -400 -0.48  83800  84200  82200  25419886  2113624966100   
2021-02-18  82100 -1100 -1.32  83200  83600  82100  21327683  1762033944231   
2021-02-17  83200 -1700 -2.00  83900  84200  83000  18307735  1526409421172   
2021-02-16  84900   700  0.83  84500  86000  84200  20483100  1740792201903   
2021-02-15  84200  2600  3.19  838

c:\Users\sms20\miniconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
